In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

np.random.seed(2018)

In [2]:
trn = pd.read_csv('data/train_ver2.csv')

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tst = pd.read_csv('data/test_ver2.csv')

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
use_dates = ['2015-05-28', '2015-06-28', '2016-05-28']

In [5]:
trn = trn[trn['fecha_dato'].isin(use_dates)]

In [6]:
prods = trn.columns[24:].tolist()

In [7]:
for col in trn.columns[24:]:
    tst[col] = 0.0
    
# merge train and test
df = pd.concat([trn, tst], axis=0)

In [8]:
# fillna for target values
df[prods] = df[prods].fillna(0.0).astype(np.int8)

In [81]:
features = []

In [84]:
df.columns[:24].tolist()

['fecha_dato',
 'ncodpers',
 'ind_empleado',
 'pais_residencia',
 'sexo',
 'age',
 'fecha_alta',
 'ind_nuevo',
 'antiguedad',
 'indrel',
 'ult_fec_cli_1t',
 'indrel_1mes',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'cod_prov',
 'nomprov',
 'ind_actividad_cliente',
 'renta',
 'segmento']

In [10]:
# data cleaning
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 
                'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']

for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
    
features += categorical_cols

In [11]:
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [12]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [14]:
df.fillna(-99, inplace=True)

In [ ]:
df['fecha_dato'].unique()

In [16]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] # "2016-05-28"
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [17]:
df.loc[df['fecha_dato'] == '2015-05-28', 'int_date'] += 1

In [18]:
df.loc[df['fecha_dato'] == '2016-05-28', 'int_date'] += 1

In [19]:
df[['int_date','fecha_dato']].iloc[100000]

int_date               6
fecha_dato    2015-05-28
Name: 2612427, dtype: object

In [20]:
df_june = df[df['fecha_dato'].isin(['2015-06-28', '2016-06-28'])]

In [21]:
df_june.shape

(1561725, 53)

In [22]:
df_may = df[df['fecha_dato'].isin(['2015-05-28', '2016-05-28'])]

In [23]:
df_may.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df_may.columns ]

In [24]:
df_may.head()

,fecha_dato_prev,ncodpers,ind_empleado_prev,pais_residencia_prev,sexo_prev,age_prev,fecha_alta_prev,ind_nuevo_prev,antiguedad_prev,indrel_prev,...,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,fecha_alta_year_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev,int_date
2512427,2015-05-28,1061260,0,0,0,24,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,9,2012,0,0,6
2512428,2015-05-28,1061283,0,0,0,22,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,9,2012,0,0,6
2512429,2015-05-28,1061284,0,0,0,24,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,9,2012,0,0,6
2512430,2015-05-28,1061336,0,0,0,23,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,9,2012,0,0,6
2512431,2015-05-28,1061286,0,0,1,29,2012-09-17,0.0,34,1.0,...,0,0,0,0,0,9,2012,0,0,6


In [25]:
df_trn = df_june.merge(df_may, on=['ncodpers','int_date'], how='left')

In [26]:
df_trn.shape

(1561725, 104)

In [89]:
features += [feature + '_prev' for feature in features]

In [90]:
features += [prod + '_prev' for prod in prods]

In [28]:
df_trn['fecha_dato'].unique()

array(['2015-06-28', '2016-06-28'], dtype=object)

In [29]:
trn = df_trn[df_trn['fecha_dato'] == '2015-06-28']
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']

In [30]:
trn.shape, tst.shape

((632110, 104), (929615, 104))

In [32]:
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)

In [39]:
X = pd.concat(X)

In [40]:
Y = np.hstack(Y)

In [45]:
X['y'] = Y

In [48]:
mask = np.random.rand(len(X)) < 0.9
x_trn = X[mask]
x_tst = X[~mask]

mask = np.random.rand(len(x_trn)) < 0.9
x_vld = x_trn[~mask]
x_trn = x_trn[mask]

In [50]:
ncodpers_tst = x_tst.as_matrix(columns=['ncodpers'])

In [51]:
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    x_tst[padd] = x_tst[prod] - x_tst[prev]

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [57]:
add_tst = x_tst.as_matrix(columns=[prod + '_add' for prod in prods])
add_tst_list = [list() for i in range(len(ncodpers_tst))]

In [59]:
count_tst = 0
for ncodper in range(len(ncodpers_tst)):
    for prod in range(len(prods)):
        if add_tst[ncodper, prod] > 0:
            add_tst_list[ncodper].append(prod)
            count_tst += 1

In [64]:
mapk(add_tst_list, add_tst_list, 7, 0.0)

1.0

In [63]:
import numpy as np

def apk(actual, predicted, k=7, default=0.0):
    # MAP@7 이므로, 최대 7개만 사용한다
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # 점수를 부여하는 조건은 다음과 같다 :
        # 예측값이 정답에 있고 (‘p in actual’)
        # 예측값이 중복이 아니면 (‘p not in predicted[:i]’) 
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # 정답값이 공백일 경우, 무조건 1.0점을 반환한다
    if not actual:
        return default

    # 정답의 개수(len(actual))로 average precision을 구한다
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list인 정답값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산한다
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)])

In [68]:
import xgboost as xgb

param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

In [94]:
X_trn = x_trn.as_matrix(columns=features)
Y_trn = x_trn.as_matrix(columns=['y'])
trn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

In [95]:
X_vld = x_vld.as_matrix(columns=features)
Y_vld = x_vld.as_matrix(columns=['y'])
vld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

In [102]:
watch_list = [(trn, 'train'), (vld, 'eval')]
model = xgb.train(param, trn, num_boost_round=1pickle.dump(model, open("next_multi.pickle", "wb"))00, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.8257	eval-mlogloss:2.82864
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.59859	eval-mlogloss:2.60675
[2]	train-mlogloss:2.42218	eval-mlogloss:2.43573
[3]	train-mlogloss:2.29052	eval-mlogloss:2.30855
[4]	train-mlogloss:2.17791	eval-mlogloss:2.20024
[5]	train-mlogloss:2.08117	eval-mlogloss:2.10801
[6]	train-mlogloss:2.00098	eval-mlogloss:2.03118
[7]	train-mlogloss:1.93242	eval-mlogloss:1.96584
[8]	train-mlogloss:1.87462	eval-mlogloss:1.91181
[9]	train-mlogloss:1.82025	eval-mlogloss:1.86053
[10]	train-mlogloss:1.76975	eval-mlogloss:1.81351
[11]	train-mlogloss:1.72707	eval-mlogloss:1.77381
[12]	train-mlogloss:1.68713	eval-mlogloss:1.73698
[13]	train-mlogloss:1.64976	eval-mlogloss:1.70255
[14]	train-mlogloss:1.61745	eval-mlogloss:1.67298
[15]	train-mlogloss:1.58636	eval-mlogloss:1.64471
[16]	train-mlogloss:1.55911	eval-mlogloss:1.61998
[17]	train

In [103]:
best_ntree_limit = model.best_ntree_limit

In [104]:
best_ntree_limit

93

In [108]:
x_trnvld = pd.concat([x_trn, x_vld])
X_trnvld = x_trnvld.as_matrix(columns=features)
Y_trnvld = x_trnvld.as_matrix(columns=['y'])
trnvld = xgb.DMatrix(X_trnvld, label=Y_trnvld, feature_names=features)

In [109]:
best_ntree_limit = int(best_ntree_limit * (len(X_trn) + len(X_vld)) / len(X_trn))

In [112]:
evallist = [(trnvld, 'trnvld')]
model = xgb.train(param, trnvld, best_ntree_limit, evals=evallist)

[0]	trnvld-mlogloss:2.82502
[1]	trnvld-mlogloss:2.61668
[2]	trnvld-mlogloss:2.43481
[3]	trnvld-mlogloss:2.3068
[4]	trnvld-mlogloss:2.18805
[5]	trnvld-mlogloss:2.09779
[6]	trnvld-mlogloss:2.01436
[7]	trnvld-mlogloss:1.93938
[8]	trnvld-mlogloss:1.87724
[9]	trnvld-mlogloss:1.81988
[10]	trnvld-mlogloss:1.76984
[11]	trnvld-mlogloss:1.72448
[12]	trnvld-mlogloss:1.68384
[13]	trnvld-mlogloss:1.64671
[14]	trnvld-mlogloss:1.61355
[15]	trnvld-mlogloss:1.58312
[16]	trnvld-mlogloss:1.55546
[17]	trnvld-mlogloss:1.52927
[18]	trnvld-mlogloss:1.5061
[19]	trnvld-mlogloss:1.48484
[20]	trnvld-mlogloss:1.46513
[21]	trnvld-mlogloss:1.44793
[22]	trnvld-mlogloss:1.43
[23]	trnvld-mlogloss:1.41406
[24]	trnvld-mlogloss:1.39929
[25]	trnvld-mlogloss:1.38521
[26]	trnvld-mlogloss:1.37198
[27]	trnvld-mlogloss:1.35978
[28]	trnvld-mlogloss:1.3479
[29]	trnvld-mlogloss:1.33733
[30]	trnvld-mlogloss:1.32696
[31]	trnvld-mlogloss:1.31722
[32]	trnvld-mlogloss:1.30855
[33]	trnvld-mlogloss:1.30011
[34]	trnvld-mlogloss:1.29199
[

In [113]:
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 7433)
('age', 6721)
('antiguedad', 6649)
('nomprov', 4225)
('fecha_alta_month', 3334)
('fecha_alta_year', 2980)
('canal_entrada', 2268)
('renta_prev', 2209)
('age_prev', 2064)
('antiguedad_prev', 2031)
('ind_cco_fin_ult1_prev', 1411)
('nomprov_prev', 1288)
('ind_recibo_ult1_prev', 1207)
('sexo', 1080)
('ind_cno_fin_ult1_prev', 1045)
('ind_ecue_fin_ult1_prev', 1035)
('segmento', 961)
('fecha_alta_year_prev', 845)
('fecha_alta_month_prev', 833)
('ind_tjcr_fin_ult1_prev', 765)
('ind_reca_fin_ult1_prev', 677)
('canal_entrada_prev', 670)
('ind_dela_fin_ult1_prev', 653)
('ind_ctop_fin_ult1_prev', 575)
('ind_nomina_ult1_prev', 566)
('ind_nom_pens_ult1_prev', 546)
('ind_ctpp_fin_ult1_prev', 510)
('ind_valo_fin_ult1_prev', 451)
('tiprel_1mes', 418)
('ind_actividad_cliente', 338)
('sexo_prev', 335)
('ind_fond_fin_ult1_prev', 273)
('ind_deco_fin_ult1_prev', 269)
('segmento_prev', 243)
('ind_nuevo', 240)
('indext', 240)
('ind_ctma_fin_ult1_prev', 216)
('tiprel_1mes_pr

In [124]:
X_test = x_tst.as_matrix(columns=features)
test = xgb.DMatrix(X_test, feature_names=features)

preds = model.predict(test, ntree_limit=best_ntree_limit)

In [127]:
preds

array([[  7.78798785e-05,   7.78798785e-05,   9.43195462e-01, ...,
          1.93258915e-02,   1.56924538e-02,   1.33338282e-02],
       [  1.66632788e-04,   1.66632788e-04,   5.10314465e-01, ...,
          2.05973089e-01,   2.24359468e-01,   4.38863188e-02],
       [  9.23667685e-05,   9.23667685e-05,   9.00410891e-01, ...,
          1.91288050e-02,   3.27016674e-02,  -9.99781565e-01],
       ..., 
       [  1.51880449e-04,   1.51880449e-04,  -9.99166709e-01, ...,
          1.90527495e-02,   2.23453622e-02,   6.51772380e-01],
       [  2.10584447e-04,   2.10584447e-04,  -9.99095550e-01, ...,
          1.63232982e-02,   3.91644388e-02,   7.39915073e-01],
       [  1.41450699e-04,   1.41450699e-04,  -9.99374427e-01, ...,
          1.80670395e-02,   2.40509938e-02,   6.97778642e-01]])

In [126]:
# subtract prev1 because possessed item cannot be bought
preds = preds - x_tst.as_matrix(columns=[prod + '_prev' for prod in prods])

In [128]:
ncodpers_tst.shape

(4064, 1)

In [131]:
result = []

for ncodper, pred in zip(ncodpers_tst, preds):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result.append([ip for y,p,ip in y_prods])
    y_prods = [p for y,p,ip in y_prods]

In [134]:
preds.shape

(4064, 24)

In [136]:
mapk(add_tst_list, result, 7, 0.0)

0.79514196467629039